In [1]:
import networkx as nx
from networkx.algorithms import community #  
import torch
import itertools
import matplotlib.pyplot as plt
import csv
from networkx.algorithms.community.quality import modularity
import random
from keybert import KeyBERT
import time
from gensim import models,corpora
from math import log, exp
import json
import gensim
import pandas as pd
from gensim import models,corpora
from gensim.models import doc2vec, ldamodel
from gensim.models import CoherenceModel
import random
from gensim.models import Word2Vec
import os
import time
import tqdm


In [2]:
def readData(datelist):
    doclist=[]
    src='../../../推特GPT/GPT.csv'
    with open(src,encoding="utf-8") as f:
         f_csv = csv.reader(f)
         pos = 1
         for row in f_csv:
            if  len(row)>=6:  #控制聚类文本数目并处理缺失数据,全部跑太多了跑不了
                #print(row[0][:10])  #取到日期
                if row[0][:10] in datelist: #判断时间
                    #print(row[0])
                    dic={}
                    dic['like_count']=eval(row[4])
                    dic['retweet_count']=eval(row[5])
                    dic['data'] = row[2]
                    doclist.append(dic)
                    pos+=1                 
    return doclist

In [3]:
#工具函数 计算pmi
#已经改成了计算频率
def GetPointwiseMutualValue(word_1,word_2,doclist):
    value_1=0
    value_2=0
    value_mutual=0
    count_word=1  #一共出现多少词,加一不影响结果，只是防止除0
    for item in doclist:
        item = item['data']
        if word_1 in item: 
            value_1+=1
        if word_2 in item: 
            value_2+=1
        if word_1 in item and word_2 in item: 
            value_mutual+=1
            #print(word_1)
            #print(word_2)
            #print(item)
        count_word=count_word+item.count(' ')+1 #用空格作为分词的依据
    #value_1= value_1/count_word
    #value_2= value_2/count_word  
    #value_mutual=value_mutual/count_word
    if value_mutual != 0:
        #value_pmi=log((value_mutual/(value_1*value_2)),2)   #原始pmi
        value_pmi=value_mutual/(max(value_1,value_2))
    else:
        value_pmi=0 
    return value_pmi    

In [4]:
#工具函数算边
def GetBrim(list_word):
    #获取节点间的边信息，主要使用相似度
    list_link_dic=[]
    global threshold_similarity  #全局变量，边的门限
    model = models.word2vec.Word2Vec.load('word2vec.model') 
    for i in range(0,len(list_word)):
        dic={}
        for j in range(i+1,len(list_word)):
            try:
               similarity = model.wv.similarity(list_word[i],list_word[j])
            except:
               similarity=0   #主要是不存在某个词的情况
            if(similarity>=0.8):  #判断点的权重,感觉不判断，只取边的前多少条反而比较好
                dic={}
                dic['Source']=i
                dic['Target']=j
                dic['Type']=0
                dic['Weight']=similarity
                #print(f"边为{dic}")
                list_link_dic.append(dic)
    print('\n完成算边')
    list_link_dic = sorted(list_link_dic, key=lambda x: x['Weight'],reverse=True)
    return list_link_dic   #取前面的边

In [5]:
#工具函数 选择相关文档
def enlargeWord(event_word,doclist):
    global proportion,data_source
    for item in event_word:
      word_list=item['word']
      doc_list=[]
      #print(word_list)
      for doc in doclist:
          text = doc['data'] 
          text = [word for word in text.split() if word.strip('#') != "" or word.strip('@') != ""]  # 以空格为单位分词 
          count=0   #统计词数
          for single_word in text:
              if(single_word in word_list): 
                  count+=1
          if(count/len(word_list)>=proportion and word_list[0] in text):  #占比超过多少的视为相关文档,初始化设置为0.3. 并且需要包含核心词
              #if(doc['like_count']+doc['retweet_count']*3>=5): #过滤无用户行为的
                    doc_list.append(doc)
      item['doc']= doc_list
    return  event_word   

In [6]:
#工具函数  过滤事件
def filteEvent(event_final):
    global filter_word
    event_return=[]
    for item in event_final:
        flag=0
        #print('开始过滤')
        #print(item['word'])
        for word in item['word']:
            #print(word)
            if (word  in filter_word):
                flag=1
        if(flag==0):
            event_return.append(item)
    return event_return     

In [7]:
def getKeyWord(doclist):
    global data_source,delete_word,word_number,canditate_word
    word_list_now=[]
    kw_model = KeyBERT(model='all-mpnet-base-v2')
    list_word=[]
    list_weight=[]
    result={}
    for doc in doclist:
        item = doc['data']   
        keywords_phrase = kw_model.extract_keywords(item, 
                                     keyphrase_ngram_range=(1,1),   #这个就是范围,可以考虑词组,本实验暂时只考虑词
                                     stop_words='english', 
                                     highlight=False, 
                                     top_n=word_number) 
        keywords_phrase_list= list(keywords_phrase)#结果，返回的是列表包含元组的形式[(),()]
        #print(keywords_phrase_list)
        
        for word in keywords_phrase_list:
            #调整每个文件的屏蔽词
            #if(word[0] not in filter_word):
              if word[0] not in list_word:
                list_word.append(word[0])
                list_weight.append(word[1])
              else:
                index=list_word.index(word[0])
                list_weight[index]=list_weight[index]+word[1]
    result['word']=list_word
    result['weight']=list_weight
    sorted_word = sorted(result['word'], key=lambda x: result['weight'][result['word'].index(x)],reverse=True)[:canditate_word]  #根据weight操作word排序
    sorted_weight=sorted(result['weight'],reverse=True)[:canditate_word]

    #print(f"提取的关键词为{sorted_word}")
    word_list=[]
    for i in range(len(sorted_word)):
        dic={}
        dic['id']=i
        dic['word']=sorted_word[i]
        dic['weight']=sorted_weight[i]
        word_list.append(dic)     #候选词的列表，每个元素为字典          
    list_link_dic=GetBrim(sorted_word)   

    #print(f"边的数量为{len(list_link_dic)}")
    #聚类
    G = nx.Graph()
    for word in word_list:
        G.add_node(word['id'],size = word['weight'])
    for link in list_link_dic:   
        G.add_edge(link['Source'],link['Target'],weight=link['Weight'])
        #print(f"边为{link}")  
    communities = nx.algorithms.community.girvan_newman(G)
    best_communities = None
    best_modularity = -1
    for community in communities: #获取最佳聚类结果
        #print(community)
        Q = modularity(G, community)  #用以衡量聚类效果的函数
        if Q > best_modularity:
            best_modularity = Q 
            best_communities = community 
    list_community=[]  #存所有簇    
    list_derail=[]  #存离群点
    for i, community in enumerate(best_communities):
        node=list(community)#字典转list
        if len(community)==1:   #对于离群点
            list_derail.append(node[0])
        if len(community)<=3 and len(community)>1:   #对于小群
            list_community.append(node)
        if len(community)>3:
            list_community.append(node)
    #list_community.append(list_derail)    #最后一个一定是离群点集合
    #print(list_community)
    for event in list_community:
        ls_event_word=[]
        flag=0   #标记type
        for i in event:  #遍历事件中每一个单词
            for word in  word_list:
                if(word['id']==i): 
                    word['type']=flag  #为每个词标记所属事件
                    ls_event_word.append(word['word'])
        flag+=1            
        #print(f"当前type关键词:{ls_event_word}")
    #为离群的词添加
    derail_value = 1001
    for i in list_derail:
        for word in  word_list:
                if(word['id']==i): 
                    word['type']=derail_value  #为每个词标记所属事件
    return  word_list                          


In [8]:
def getEvent(word_list,doclist):
    global data_source,delete_word,word_number
    derail_value = 1001
    event_word=[]
    for i in range(len(word_list)):
        for j in range(i+1,len(word_list)):
            #同话题，不考虑离群
            if(word_list[i]['type']==word_list[j]['type'] and word_list[i]['type']!=derail_value):
                value_pmi=GetPointwiseMutualValue(word_list[i]['word'],word_list[j]['word'],doclist)
                if(value_pmi>=0.1):    #话题增益
                    keyword = max([word_list[i], word_list[j]], key=lambda x: x['weight'])
                    if(keyword not in event_word):
                        event_word.append(keyword)  #存核心词,然后再看第二步
    event_final=[]
    keyword_list=[]
    for keyword in event_word:
        keyword_list.append(keyword['word'])
    for keyword in event_word:
        dic={}
        dic['keyword']=keyword
        #print(f'核心词{keyword}')
        dic['word']=[]
        dic['word'].append(keyword['word'])#先把keyword本身的词加进去
        dic['flag']=0
        for word in word_list:
            #if(word['word'] not in keyword_list):
            if(word['word'] != keyword['word']):
                pmi = GetPointwiseMutualValue(keyword['word'],word['word'],doclist)
                if(pmi>=0.05 ):  #看pmi值
                    if(keyword['type'] == word['type'] or word['type'] == derail_value):   #存一个type的 
                        #print(keyword['word'],keyword['type'],word['word'],word['type'],pmi)
                        dic['word'].append(word['word'])
                        #print(word['word'])
                        #print(pmi)
        event_final.append(dic)                 
    event_final = enlargeWord(event_final,doclist)

    #处理可能存在的重复问题。  如果一个事件的keyword在另一个事件的word中，则合并
    event_final_empty=[]
    for i in range(len(event_final)):
        #print(event_final[i])
        for j in range(i+1,len(event_final)):
            if(event_final[i]['keyword']['word'] in event_final[j]['word'] or event_final[j]['keyword']['word'] in event_final[i]['word']):   #判断keyword在word列表中的情况
                event_final[i]['word']=list(set(event_final[i]['word']+event_final[j]['word']))
                event_final[j]['flag']=1
        if(event_final[i]['flag']==0):
           event_final_empty.append(event_final[i])
    event_final = event_final_empty
    #过滤event
    event_final = filteEvent(event_final)
    event_return=[]
    for item in event_final:
        if(len(item['doc'])>5):
            event_return.append(item)
    return  event_return        

In [9]:
#工具函数，获得全局需要删除的高频词，只运行一次
#需要记录结果
def getGloablWord():
    global delete_word,word_number
    kw_model = KeyBERT(model='all-mpnet-base-v2')
    doclist=[]
    with open('../推特GPT/GPT.csv',encoding="utf-8") as f:
         f_csv = csv.reader(f)
         next(f_csv)
         pos = 1
         for row in f_csv:
            if  len(row)>=6:  #控制聚类文本数目并处理缺失数据,全部跑太多了跑不了
                #print(row[0][:10])  #取到日期
                    doclist.append(row[2])
                    pos+=1
         print(f'总数量{pos}')
    list_word=[]
    list_weight=[]
    result={}     
    progress_bar=tqdm.tqdm(total=len(doclist))
    for doc in doclist:
        item = doc 
        keywords_phrase = kw_model.extract_keywords(item, 
                                     keyphrase_ngram_range=(1,1),   #这个就是范围,可以考虑词组,本实验暂时只考虑词
                                     stop_words='english', 
                                     highlight=False, 
                                     top_n=word_number) 
        keywords_phrase_list= list(keywords_phrase)#结果，返回的是列表包含元组的形式[(),()]
        #print(keywords_phrase_list)
        
        for word in keywords_phrase_list:
          if word[0] not in delete_word[:10]:         
              if word[0] not in list_word:
                list_word.append(word[0])
                list_weight.append(word[1])
              else:
                index=list_word.index(word[0])
                list_weight[index]=list_weight[index]+word[1]
        progress_bar.update(1)        
    result['word']=list_word
    result['weight']=list_weight
    sorted_word = sorted(result['word'], key=lambda x: result['weight'][result['word'].index(x)],reverse=True)  #根据weight操作word排序
    sorted_weight=sorted(result['weight'],reverse=True)
   
    #print(sorted_word)
    #print(sorted_weight)
    return sorted_word,sorted_weight

In [10]:
filter_word=['chatgpt', 'gpt', 'ai', 'chat', 'openai', 'artificialintelligence', 'chatbot', 'google', 'bing', 'https','data','language']
#filter_word=[]  #不要全局的增益
proportion = 0.4
#记录一个问题，这种数据如果训练词向量的时候维度太高，就不好用相似度解开
random.seed(42)
#date_list=['2022-01-01','2022-01-02','2022-01-03','2022-01-04','2022-01-05']
date_list=[
    ['2023-01-06','2023-01-07','2023-01-08'],
    ['2023-01-09','2023-01-10','2023-01-11'],
    ['2023-01-12','2023-01-13','2023-01-14'],
    ['2023-01-15','2023-01-16','2023-01-17'],
    ['2023-01-18','2023-01-19','2023-01-20'],
    ['2023-01-21','2023-01-22','2023-01-23'],
    ['2023-01-24','2023-01-25','2023-01-26'],
    ['2023-01-27','2023-01-28','2023-01-29'],
    ['2023-01-30','2023-01-31','2023-02-01'],
    ['2023-02-02','2023-02-03','2023-02-04'],
    ['2023-02-05','2023-02-06','2023-02-07'],
    ['2023-02-08','2023-02-09','2023-02-10'],
    ['2023-02-11','2023-02-12','2023-02-13'],
    ['2023-02-14','2023-02-15','2023-02-16'],
    ['2023-02-17','2023-02-18','2023-02-19'],
    ['2023-02-20','2023-02-21','2023-02-22'],
    ['2023-02-23','2023-02-24','2023-02-25'],
    ['2023-02-26','2023-02-27','2023-02-28'],
]
word_number=3
canditate_word = 60

In [11]:
real_event_list=['education', 'teachers','student','students','teacher','school', 'writing', 'write','schools', 'essay','essays','tweet', 'tweets','bitcoin', 'crypto','prompts', 'prompt'
'cybersecurity', 'malware','exam', 'exams','bias', 'biased','marketing', 'business','market', 'phishing','pish','generated','jobs','job','billion','worth','value','poem','art']
real_event_word=[
    ['students','teachers','schools','education','NYC'],
    ['student','teacher','school','education','NYC'],
    ['tweet','generated'],
    ['write','essay','poem','art'],
    ['cybersecurity','pish','malware'],
    ['exam'],
    ['bias'],
    ['valentine'],
    ['jobs','replace'],
    ['bitcoin','crypto','marketing','business'],
    ['write','prompt'],
    ['billion','worth','value']
]
real_event_length=27
result=[]

In [12]:
start_time = time.time()
word_list_pre=[]
for item in date_list:
        print(f"日期为{item}")
        doclist=readData(item)
        #print(len(doclist))
        word_list=getKeyWord(doclist)
        word_list_now=word_list.copy()  #用以暂存这次的,注意深复制
        for word_pre in word_list_pre:
            flag=0
            for item in word_list:
                if(item['word']==word_pre['word']):
                    flag=1
            if(flag==0):
                word_list.append(word_pre)
        print(f"词总数为{len(word_list)}")        
        event_final=getEvent(word_list,doclist)
        for event in event_final[:50]:
               #print(event['keyword']['word'],event['word'])
                print(event['word'])
                result.append(event['word'])
                #for doc in event['doc'][:50]:
                #    print(doc)

        word_list_pre = word_list_now            
end_time = time.time()                

日期为['2023-01-06', '2023-01-07', '2023-01-08']

完成算边
词总数为60
['essay', 'education', 'writing', 'teachers', 'students', 'schools']
['tweet', 'tweets']
['startups', 'youtube', 'valuation', 'automation']
日期为['2023-01-09', '2023-01-10', '2023-01-11']

完成算边
词总数为74
['writing', 'text', 'code', 'essay', 'content', 'write']
['marketing', 'seo', 'business', 'content']
['essay', 'education', 'essays', 'twitter', 'students', 'teachers', 'schools']
['tweet', 'tweets']
日期为['2023-01-12', '2023-01-13', '2023-01-14']

完成算边
词总数为71
['prompts', 'prompt']
['tweet', 'tweets']
日期为['2023-01-15', '2023-01-16', '2023-01-17']

完成算边
词总数为74
['code', 'writing', 'nick', 'content', 'search', 'write', 'song', 'prompt', 'nickcave']
['teaching', 'essay', 'education', 'essays', 'twitter', 'students', 'teachers', 'write']
['tweet', 'tweets']
['youtube', 'automation']
日期为['2023-01-18', '2023-01-19', '2023-01-20']

完成算边
词总数为73
['prompts', 'prompt']
日期为['2023-01-21', '2023-01-22', '2023-01-23']

完成算边
词总数为72
['tweet', 'tweets']

In [13]:
all_word=[]
get_real_word=[]
count_recall_event=0
for event in result:
    # 使用逗号分割字符串，并去除空格
    words = event
    print(f"获得的事件为{words}")
    for word in words:
        if(word in real_event_list):
            #print(word)
            get_real_word.append(word)
    flag_belong_event=0        
    for real_event_this in real_event_word:
        count_belong_this=0
        for real_word_this in real_event_this :        
            if(real_word_this in words):
                count_belong_this+=1
        if(count_belong_this>len(real_event_this)*0.4 and flag_belong_event==0): 
            count_recall_event+=1
            flag_belong_event =1  #置1，只能属于一个事件
            print(f"归属于事件{real_event_this}")
    all_word.extend(words) 
all_word = list(set(all_word)) 
get_real_word = list(set(get_real_word))
print(f"全部词为{all_word}")
print(f'召回词为{get_real_word}')          
print(f'事件正确率{count_recall_event/len(result)}')
print(f'召回词数{len(get_real_word)}')
print(f'召回事件')
print(f'词正确率为{len(get_real_word)/len(all_word)}')
print(f"运行时间为{end_time - start_time}")

获得的事件为['essay', 'education', 'writing', 'teachers', 'students', 'schools']
归属于事件['students', 'teachers', 'schools', 'education', 'NYC']
获得的事件为['tweet', 'tweets']
归属于事件['tweet', 'generated']
获得的事件为['startups', 'youtube', 'valuation', 'automation']
获得的事件为['writing', 'text', 'code', 'essay', 'content', 'write']
归属于事件['write', 'essay', 'poem', 'art']
获得的事件为['marketing', 'seo', 'business', 'content']
归属于事件['bitcoin', 'crypto', 'marketing', 'business']
获得的事件为['essay', 'education', 'essays', 'twitter', 'students', 'teachers', 'schools']
归属于事件['students', 'teachers', 'schools', 'education', 'NYC']
获得的事件为['tweet', 'tweets']
归属于事件['tweet', 'generated']
获得的事件为['prompts', 'prompt']
归属于事件['write', 'prompt']
获得的事件为['tweet', 'tweets']
归属于事件['tweet', 'generated']
获得的事件为['code', 'writing', 'nick', 'content', 'search', 'write', 'song', 'prompt', 'nickcave']
归属于事件['write', 'prompt']
获得的事件为['teaching', 'essay', 'education', 'essays', 'twitter', 'students', 'teachers', 'write']
归属于事件['students', 'teachers'